<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyVIRUS</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>Virus</b> modelling</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyVIRUS.png"></td>
</tr>
</table>

----
# `pyVIRUS`

pyVIRUS, a program package for for modelling the spread and containment of a virus in a population.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
import ipywidgets as widgets

----
# Evolution models (SEIR)
In this notebook, we look into virus modelling with coupled differential equations.
This setup is actually very similar to our geoscientific decay-series modeling for radiogenic isotopes!

The **SEIR** model describes how a dicease spreads in a population from *healty* individuals to *invected* persons to *immune* people. 

**SEIR** stands for *Susceptible – Exposed - Infectious – Recovered*, it is a box model. We define $N$ as
the total number of individuals.

Then the number of people *susceptible* to the infection, the number of people *exposed*,
the number of people *invected*, and the number of people *recovered and immune* are given as time functions:

- $S(t)$: Number of susceptible people [1/number]

- $E(t)$: Number of exposed people [1/number]

- $I(t)$: Number of invected people [1/number]

- $R(t)$: Number of recovered people [1/number]

with $t$ the time.

For the total number of people $N$, we define:
$$
N = S(t)+E(t)+I(t)+R(t)
$$

During the invection, people move from compartment $S$ via compartments $E$ and $I$ to compartment $R$.
This is described by the following set of **coupled ordinary differential equation of first order**:
$$
\begin{array}{rcl}
{{dS} \over {dt}} & = & - {{R_t} \over {T_{inf}}} I S, \\
{{dE} \over {dt}} & = & + {{R_t} \over {T_{inf}}} I S - {{1} \over {T_{inc}}} E, \\
{{dI} \over {dt}} & = & + {{1} \over {T_{inc}}} E - {{1} \over {T_{inf}}} I, \\
{{dR} \over {dt}} & = & + {{1} \over {T_{inf}}} I, \\
\end{array}
$$
with $R_t$ [1/number] the *transmission rate* (often called $R_0$ in the literature), 
$T_{inf}$ [days] the *incubation period*,
and $T_{inc}$ [days] the *infectious period*.

In [3]:
def rhsSEIR(t,y,Rt,Tinf,Tinc):
    S, E, I, R = y
    dSdt = -Rt/Tinf*S*I
    dEdt = Rt/Tinf*I*S - E/Tinc
    dIdt = E/Tinc - I/Tinf
    dRdt = I/Tinf
    return [dSdt, dEdt, dIdt, dRdt]

In [5]:
def runSEIR(Rt,Tinf,Tinc,tmax):
    # Total population, N.
    N = 1000
    # Initial number of infected and recovered individuals, I0 and R0.
    E0, I0, R0 = 0, 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0 - E0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    #beta, gamma = 2.2, 1./28
    # A grid of time points (in days)
    t = np.linspace(0, tmax, 5*tmax)
    # The SIR model differential equations.
    # Initial conditions vector
    y0 = [S0, E0, I0, R0]
    # Integrate the SIR equations over the time grid, t.
    solution = integrate.solve_ivp(rhsSEIR,[t[0],t[-1]],y0,t_eval=t,args=(Rt,Tinf,Tinc))
    S, E, I, R = solution.y
    # Plot the data on three separate curves for S(t), I(t) and R(t)
    plt.figure(figsize=(10.0,6.0))
    plt.xlabel('Time [days]')
    plt.ylabel('People [%]')
    plt.ylim([0,110])
    plt.plot(t, 100*S/N, 'black', alpha=0.5, lw=2, label='Susceptible')
    plt.plot(t, 100*E/N, 'b', alpha=0.5, lw=2, label='Exposed')
    plt.fill_between(t, 100*I/N, 'r', alpha=0.5, lw=2, label='Infected')
    plt.plot(t, 100*R/N, 'g', alpha=0.5, lw=2, label='Recovered')
    plt.grid(which='major', c='lightgrey', lw=1, ls=':')
    plt.legend()

In [7]:
# call interactive module
w = dict(
Rt=widgets.FloatSlider(min=0.1,max=10,step=0.1,value=2.20,description='Rt',continuous_update=False),
Tinc=widgets.FloatSlider(min=0.1,max=30,step=0.1,value=5.2,description='Tinc',continuous_update=False),
Tinf=widgets.FloatSlider(min=0.1,max=30,step=0.1,value=2.9,description='Tinf',continuous_update=False),
tmax=widgets.IntSlider(min=20,max=300,step=10,value=160,description='Days'))

output = widgets.interactive_output(runSEIR, w)
box = widgets.HBox([widgets.VBox([*w.values()]), output])
display(box)

----